In [1]:
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 545.1 kB/s eta 0:00:00


In [2]:
%load_ext sql

In [3]:
%sql sqlite://

In [4]:
%%sql

DROP TABLE IF EXISTS runners;
CREATE TABLE runners (
  "runner_id" INTEGER,
  "registration_date" DATE
);
INSERT INTO runners
  ("runner_id", "registration_date")
VALUES
  (1, '2021-01-01'),
  (2, '2021-01-03'),
  (3, '2021-01-08'),
  (4, '2021-01-15');


DROP TABLE IF EXISTS customer_orders;
CREATE TABLE customer_orders (
  "order_id" INTEGER,
  "customer_id" INTEGER,
  "pizza_id" INTEGER,
  "exclusions" VARCHAR(4),
  "extras" VARCHAR(4),
  "order_time" TIMESTAMP
);

INSERT INTO customer_orders
  ("order_id", "customer_id", "pizza_id", "exclusions", "extras", "order_time")
VALUES
  ('1', '101', '1', '', '', '2020-01-01 18:05:02'),
  ('2', '101', '1', '', '', '2020-01-01 19:00:52'),
  ('3', '102', '1', '', '', '2020-01-02 23:51:23'),
  ('3', '102', '2', '', NULL, '2020-01-02 23:51:23'),
  ('4', '103', '1', '4', '', '2020-01-04 13:23:46'),
  ('4', '103', '1', '4', '', '2020-01-04 13:23:46'),
  ('4', '103', '2', '4', '', '2020-01-04 13:23:46'),
  ('5', '104', '1', 'null', '1', '2020-01-08 21:00:29'),
  ('6', '101', '2', 'null', 'null', '2020-01-08 21:03:13'),
  ('7', '105', '2', 'null', '1', '2020-01-08 21:20:29'),
  ('8', '102', '1', 'null', 'null', '2020-01-09 23:54:33'),
  ('9', '103', '1', '4', '1, 5', '2020-01-10 11:22:59'),
  ('10', '104', '1', 'null', 'null', '2020-01-11 18:34:49'),
  ('10', '104', '1', '2, 6', '1, 4', '2020-01-11 18:34:49');


DROP TABLE IF EXISTS runner_orders;
CREATE TABLE runner_orders (
  "order_id" INTEGER,
  "runner_id" INTEGER,
  "pickup_time" VARCHAR(19),
  "distance" VARCHAR(7),
  "duration" VARCHAR(10),
  "cancellation" VARCHAR(23)
);

INSERT INTO runner_orders
  ("order_id", "runner_id", "pickup_time", "distance", "duration", "cancellation")
VALUES
  ('1', '1', '2020-01-01 18:15:34', '20km', '32 minutes', ''),
  ('2', '1', '2020-01-01 19:10:54', '20km', '27 minutes', ''),
  ('3', '1', '2020-01-03 00:12:37', '13.4km', '20 mins', NULL),
  ('4', '2', '2020-01-04 13:53:03', '23.4', '40', NULL),
  ('5', '3', '2020-01-08 21:10:57', '10', '15', NULL),
  ('6', '3', 'null', 'null', 'null', 'Restaurant Cancellation'),
  ('7', '2', '2020-01-08 21:30:45', '25km', '25mins', 'null'),
  ('8', '2', '2020-01-10 00:15:02', '23.4 km', '15 minute', 'null'),
  ('9', '2', 'null', 'null', 'null', 'Customer Cancellation'),
  ('10', '1', '2020-01-11 18:50:20', '10km', '10minutes', 'null');


DROP TABLE IF EXISTS pizza_names;
CREATE TABLE pizza_names (
  "pizza_id" INTEGER,
  "pizza_name" TEXT
);
INSERT INTO pizza_names
  ("pizza_id", "pizza_name")
VALUES
  (1, 'Meatlovers'),
  (2, 'Vegetarian');


DROP TABLE IF EXISTS pizza_recipes;
CREATE TABLE pizza_recipes (
  "pizza_id" INTEGER,
  "toppings" TEXT
);
INSERT INTO pizza_recipes
  ("pizza_id", "toppings")
VALUES
  (1, '1, 2, 3, 4, 5, 6, 8, 10'),
  (2, '4, 6, 7, 9, 11, 12');


DROP TABLE IF EXISTS pizza_toppings;
CREATE TABLE pizza_toppings (
  "topping_id" INTEGER,
  "topping_name" TEXT
);
INSERT INTO pizza_toppings
  ("topping_id", "topping_name")
VALUES
  (1, 'Bacon'),
  (2, 'BBQ Sauce'),
  (3, 'Beef'),
  (4, 'Cheese'),
  (5, 'Chicken'),
  (6, 'Mushrooms'),
  (7, 'Onions'),
  (8, 'Pepperoni'),
  (9, 'Peppers'),
  (10, 'Salami'),
  (11, 'Tomatoes'),
  (12, 'Tomato Sauce');

 * sqlite://
Done.
Done.
4 rows affected.
Done.
Done.
14 rows affected.
Done.
Done.
10 rows affected.
Done.
Done.
2 rows affected.
Done.
Done.
2 rows affected.
Done.
Done.
12 rows affected.


[]

In [ ]:
#1. How many pizzas were ordered?
%%sql
select count(*) from customer_orders;

 * sqlite://
Done.


count(*)
14


In [ ]:
#2. How many unique customer orders were made?
%%sql
select count(distinct(order_id)) from customer_orders;

 * sqlite://
Done.


count(distinct(order_id))
10


In [ ]:
#3. How many successful orders were delivered by each runner?
%%sql
select runner_id, count(*) from runner_orders where cancellation is null group by runner_id;

 * sqlite://
Done.


runner_id,count(*)
1,1
2,1
3,1


In [ ]:
#4. How many of each type of pizza was delivered?
%%sql
with succ_delivery as(
  select runner_orders.order_id,
  customer_orders.pizza_id,
  runner_orders.cancellation,
  pizza_names.pizza_name
  from runner_orders
  inner join customer_orders
  on runner_orders.order_id = customer_orders.order_id
  inner join pizza_names
  on pizza_names.pizza_id = customer_orders.pizza_id
)
select pizza_name, count(*) from succ_delivery where cancellation is null group by pizza_name;

 * sqlite://
Done.


pizza_name,count(*)
Meatlovers,4
Vegetarian,2


In [ ]:
#5. How many Vegetarian and Meatlovers were ordered by each customer?
%%sql
with pizza_count as(
  select customer_orders.customer_id,
  pizza_names.pizza_name
  from customer_orders
  join pizza_names on pizza_names.pizza_id = customer_orders.pizza_id
)
select customer_id, pizza_name, count(pizza_name) from pizza_count group by customer_id, pizza_name;

 * sqlite://
Done.


customer_id,pizza_name,count(pizza_name)
101,Meatlovers,2
101,Vegetarian,1
102,Meatlovers,2
102,Vegetarian,1
103,Meatlovers,3
103,Vegetarian,1
104,Meatlovers,3
105,Vegetarian,1


In [10]:
#6. What was the maximum number of pizzas delivered in a single order?
%%sql
with pizza_count as(
  select customer_orders.order_id,
  count(customer_orders.pizza_id) as pizza_count
  from customer_orders
  group by customer_orders.order_id
)
select max(pizza_count) from pizza_count;


 * sqlite://
Done.


max(pizza_count)
3


In [13]:
#7. For each customer, how many delivered pizzas had at least 1 change and how many had no changes?
%%sql
with changes as(
  select customer_orders.order_id,
  customer_orders.customer_id,
  customer_orders.pizza_id,
  case
    when exclusions is null or exclusions = '' then 0
    else 1
    end as exclusions_change,
  case
    when extras is null or extras = '' then 0
    else 1
  end as extras_change
  from customer_orders
  join runner_orders
  on runner_orders.order_id = customer_orders.order_id
  where cancellation is null
)
select order_id,
customer_id,
sum(exclusions_change) as exclusions_change,
sum(extras_change) as extras_change
from changes
group by order_id, customer_id;

 * sqlite://
Done.


order_id,customer_id,exclusions_change,extras_change
3,102,0,0
4,103,3,0
5,104,1,1


In [15]:
#8. How many pizzas were delivered that had both exclusions and extras?
%%sql
WITH changes AS (
  SELECT
    customer_orders.order_id,
    customer_orders.customer_id,
    customer_orders.pizza_id
  FROM customer_orders
  JOIN runner_orders ON customer_orders.order_id = runner_orders.order_id
  WHERE cancellation IS NULL
    AND customer_orders.exclusions IS NOT NULL
    AND customer_orders.extras IS NOT NULL
)
SELECT
  order_id,
  customer_id,
  COUNT(pizza_id) AS pizza_count
FROM changes
GROUP BY order_id, customer_id;


 * sqlite://
Done.


order_id,customer_id,pizza_count
3,102,1
4,103,3
5,104,1


In [21]:
#9. What was the total volume of pizzas ordered for each hour of the day?
%%sql

WITH hour_wise_count AS (
  SELECT
    customer_orders.order_id,
    customer_orders.pizza_id,
    strftime('%H', order_time) AS hour_wise_time
  FROM customer_orders
)
SELECT
  hour_wise_time,
  COUNT(pizza_id) AS pizza_count
FROM hour_wise_count
GROUP BY hour_wise_time
ORDER BY hour_wise_time;


 * sqlite://
Done.


hour_wise_time,pizza_count
11,1
13,3
18,3
19,1
21,3
23,3


In [27]:
#10. What was the volume of orders for each day of the week?
%%sql
WITH day_wise_count AS (
  SELECT
    customer_orders.order_id,
    customer_orders.pizza_id,
    strftime('%w', order_time) AS day_wise_time
  FROM customer_orders
)
SELECT
  CASE
    WHEN day_wise_time = '0' THEN 'Sunday'
    WHEN day_wise_time = '1' THEN 'Monday'
    WHEN day_wise_time = '2' THEN 'Tuesday'
    WHEN day_wise_time = '3' THEN 'Wednesday'
    WHEN day_wise_time = '4' THEN 'Thursday'
    WHEN day_wise_time = '5' THEN 'Friday'
    WHEN day_wise_time = '6' THEN 'Saturday'
  END AS day_name,
  COUNT(pizza_id) AS pizza_count
FROM day_wise_count
GROUP BY day_name
ORDER BY day_wise_time;


 * sqlite://
Done.


day_name,pizza_count
Wednesday,5
Thursday,3
Friday,1
Saturday,5


# **Runner and Customer Experience**

In [7]:
#1. How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)
%%sql
with weekly_runn_regs as(
  select
  runners.runner_id,
  strftime('%W', registration_date) AS week_of_reg
  from runners
)
select
week_of_reg,
count(runner_id)
from weekly_runn_regs
group by week_of_reg
order by week_of_reg;

 * sqlite://
Done.


week_of_reg,count(runner_id)
00,2
01,1
02,1


In [10]:
#2.What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?
%%sql
with time_taken as(
  select customer_orders.order_time,
  customer_orders.order_id,
  runner_orders.runner_id,
  runner_orders.pickup_time,
  (julianday(runner_orders.pickup_time) - julianday(customer_orders.order_time)) * 1440 AS time_taken_minutes
  from customer_orders
  join
  runner_orders
  on customer_orders.order_id = runner_orders.order_id
)
select
order_id,
runner_id,
avg(time_taken_minutes) as avg_time
from time_taken
group by order_id
order by order_id;

 * sqlite://
Done.


order_id,runner_id,avg_time
1,1,10.533333197236061
2,1,10.033333897590637
3,1,21.233333498239517
4,2,29.283332750201225
5,3,10.466666892170906
6,3,None
7,2,10.266666635870934
8,2,20.483333542943
9,2,None
10,1,15.516666322946548


In [11]:
#3. Is there any relationship between the number of pizzas and how long the order takes to prepare?
%%sql
with num_piz as(
  select customer_orders.order_id,
  count(customer_orders.pizza_id) as pizza_count,
  (julianday(runner_orders.pickup_time) - julianday(customer_orders.order_time)) * 1440 AS time_taken_minutes
  from customer_orders
  join
  runner_orders
  on customer_orders.order_id = runner_orders.order_id
  group by customer_orders.order_id
)
select pizza_count,
avg(time_taken_minutes) as avg_time
from num_piz
group by pizza_count
order by pizza_count;

 * sqlite://
Done.


pizza_count,avg_time
1,12.356666833162308
2,18.374999910593033
3,29.283332750201225


In [12]:
#4.What was the average distance travelled for each customer?
%%sql
with dist_trvl as
(
  select customer_orders.customer_id,
  runner_orders.distance
  from customer_orders
  join
  runner_orders
  on customer_orders.order_id = runner_orders.order_id
)
select
customer_id,
avg(distance) as avg_dist_travelled
from dist_trvl
group by customer_id
order by customer_id;

 * sqlite://
Done.


customer_id,avg_dist_travelled
101,13.333333333333334
102,16.733333333333334
103,17.549999999999997
104,10.0
105,25.0


In [18]:
#5. What was the difference between the longest and shortest delivery times for all orders?
%%sql
with time_taken as(
  select customer_orders.order_id,
  julianday(runner_orders.pickup_time) - julianday(customer_orders.order_time) *1440 AS time_taken_minutes
  from customer_orders
  join
  runner_orders
  on customer_orders.order_id = runner_orders.order_id
)
select MAX(time_taken_minutes) - MIN(time_taken_minutes) as diff_time
from time_taken;

 * sqlite://
Done.


diff_time
14419.759190559387


In [28]:
#6.What was the average speed for each runner for each delivery and do you notice any trend for these values?
%%sql
WITH time_taken AS (
  SELECT
    customer_orders.order_id,
    runner_orders.runner_id,
    runner_orders.duration / 60.0 AS duration_hours,
    runner_orders.distance
  FROM
    customer_orders
  JOIN
    runner_orders
  ON customer_orders.order_id = runner_orders.order_id
)
SELECT
  runner_id,
  AVG(distance / duration_hours) AS avg_speed
FROM time_taken
GROUP BY runner_id
ORDER BY runner_id;

 * sqlite://
Done.


runner_id,avg_speed
1,47.0574074074074
2,51.779999999999994
3,40.0


In [32]:
#7. What is the successful delivery percentage for each runner?
%%sql
WITH succ_delivery AS (
  SELECT
    runner_orders.order_id,
    runner_orders.runner_id
  FROM runner_orders
  WHERE runner_orders.cancellation IS NULL or runner_orders.cancellation = ''
),
total_orders AS (
  SELECT COUNT(order_id) AS total_count
  FROM runner_orders
)
SELECT
  runner_id,
  COUNT(order_id) AS total_successful_orders,
  (COUNT(order_id) * 100.0 / (SELECT total_count FROM total_orders)) AS successful_delivery_percentage
FROM succ_delivery
GROUP BY runner_id;


 * sqlite://
Done.


runner_id,total_successful_orders,successful_delivery_percentage
1,3,30.0
2,1,10.0
3,1,10.0
